In [ ]:
'''
Alberto Mangones
Alberto.Mangones@groupm.com
Feb 2019
 
Audience Segment Analysis
Filter out only top x based on either CTR, Conversion Rate or Imps vol
'''
import pandas as pd
import os
import ftplib
from ftplib import FTP
import datetime
from io import BytesIO, StringIO
import datorama

#params
agency = "Mediacom"
min_vol = 999
ftp_path = '/Insights/Audiences/' + agency
ftp_ref = '/Insights/Audiences/Ref'
ftp_out_path = '/Insights/Audiences/'+agency+'/Datorama'
ftp_pw = 'Ca125XaX'
ftp_server = 'analytics.xaxis.com'
ftp_user = 'ftp_analyticscanada'
f_in_keyword = 'DBM_Audience_Segments_2019_Mediacom' #report name or partial name
skip_footer_row_count = 15 # the current report has 15 rows to exclude at the end, update accordingly
f_ref_keyword = 'DBM_MCM_Audience_Campaign_Ref' # keywords to pickup reference file from Datorama


#Calculating the date of the run
date = datetime.datetime.now().strftime("%Y-%m-%d")

def get_ftp_file(server_path, ftp_report_name=None, delete_files=False):
    """Retrieves  file from ftp server and returns it as a blob object.
    Args:
        server_path - folder where to get the file
        ftp_report_name - keywords used in case there are more than one report in the given folder
        delete_files - flag to delete files after retrieving
    Returns:
        blob object
    """
    file_object= BytesIO()
    ftp_path = server_path
    try:
        ftp = FTP(host=ftp_server, user=ftp_user, passwd=ftp_pw)
        ftp.cwd(server_path)

        # Get list and pick most recent file
        ftp_files = ftp.nlst()
        if ftp_report_name:
            ftp_files = [x for x in ftp_files if ftp_report_name in x]
        ftp_files.sort(reverse=True)

        if len(ftp_files) > 0:
            ftp_file = ftp_files[0]
            ftp.retrbinary('RETR ' + ftp_file, file_object.write)
            datorama.log(os.getcwd())
        else:
            # No files to extract
            datorama.log("No files in ftp folder")
            return None
    except ftplib.all_errors:
        # Error occured - nothing to do
        datorama.log('Spotlight FTP - Unable to access FTP file')
        return None

    if delete_files:
        # Do some cleanup in the ftp folder
        ftp_files = ftp.nlst()
        for i in ftp_files:
            try:
                ftp.delete(i)
            except ftplib.all_errors:
                datorama.log('Unable to delete files from ftp')
                pass
            
    return file_object

def put_ftp(server_path, localfile, blob):
    """ Uploads file to ftp server
    Args:
        server_path - folder where to store the file in the ftp folder
        localfile - if the file is stored locally, direct path to file
        blob - if the file is in memory, blob pointer to the object
    Returns:
        None
    """
    ftp = FTP(host=ftp_server, user=ftp_user, passwd=ftp_pw)
    ftp.cwd(server_path)
    f_name = os.path.basename(localfile)
    if blob is None:
        with open(localfile, 'rb') as f:
            ftp.storbinary('STOR ' + f_name, f)
    else:
        ftp.storbinary('STOR ' + f_name, blob)
    ftp.quit()
    
# read file
f_in = get_ftp_file(ftp_path, f_in_keyword)
df = pd.read_excel(f_in, skipfooter=skip_footer_row_count)

# Use this if reading csv 
#df = pd.read_csv(StringIO(f_in.getvalue().decode('utf-8')))

f_in = get_ftp_file(ftp_ref, f_ref_keyword)
df_ref = pd.read_csv(StringIO(f_in.getvalue().decode('utf-8')))
df_ref = df_ref[['Campaign Name','DBM Campaign Name','SF_Start_Opportunity']]

'''
Columns:
       u'Advertiser Currency', u'Advertiser', u'Insertion Order',
       u'Audience List ID', u'Audience List', u'Audience List Type',
       u'Impressions', u'Clicks', u'Total Conversions',
       u'Complete Views (Video)'
'''

# Filter out empty lines and lines and are not from DBM
df = df[df['Advertiser Currency']=='CAD']
df = df[df['Audience List Type'] == 'Bid Manager Audiences']
df['Audience List ID'] = df['Audience List ID'].astype(str)

# Clean Up Audience Lists
seg_keyword = r'Display & Video 360 Audiences \xbb '
df['Audience List'] = df['Audience List'].str.replace(seg_keyword, '')
cols = ['Type', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6']
df[cols]= df['Audience List'].str.split(pat=u'\xbb\s+', expand=True)

# Get rid of blank spaces
for t in cols:
    df[t] = df[t].str.strip()
    
# Further aggregate from T1-T3
df['T2'] = df.T2.fillna(' ', axis=0)
df['T3'] = df.T3.fillna(' ', axis=0)
df['T4'] = ''
df['T5'] = ''
df['T6'] = ''

# Merging with the ref df
#join with audience data
df = pd.merge(df, df_ref, how='left', right_on='Campaign Name', left_on='Insertion Order')
start_date = df.sample()['SF_Start_Opportunity'].tolist()[0]
campaign_name = df.sample()['DBM Campaign Name'].tolist()[0]

group_by_cols = ['Advertiser','Campaign Name', 'Type','T1','T2','T3']
dfg = df.groupby(group_by_cols, as_index=False).sum()
dfg['T2'] = dfg['T2'].str.title()
dfg['Audience List ID'] = dfg.index.astype(str)
dfg['Audience Name'] = dfg.Type +'_' + dfg.T1+'_' + dfg.T2 + '_' + dfg.T3

# Handling negative values for Impressions, Clicks
dfg['Impressions'] = dfg.Impressions.apply(lambda x: max(0, x))
dfg['Clicks'] = dfg.Clicks.apply(lambda x: max(0, x))

# Get rid of low volume segments
dfg = dfg[dfg.Impressions > min_vol]

# Add the Salesforce Opportunity Start Date for reference
dfg['Date'] = start_date
dfg.rename(columns={'Campaign Name':'Campaign'}, inplace=True)

In [4]:
cols

['Type', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']

In [6]:
csv_string = dfg.to_csv(index=False)
df_blob = StringIO(csv_string.decode(''))
ftp_out_file = campaign_name + '.csv'

put_ftp(ftp_out_path, ftp_out_file, df_blob)

UnicodeEncodeError: 'ascii' codec can't encode character u'\xc2' in position 30: ordinal not in range(128)

In [43]:
csv_string.shape

AttributeError: 'function' object has no attribute 'shape'

In [8]:
''' 
# To save locally uncomment the code below
env = os.environ.get('HOMEPATH')
local_filename = os.path.os.path.join(env, campaign_name+'.csv')
dfg.to_csv(local_filename, index=False)

'''

UnicodeEncodeError: 'ascii' codec can't encode character u'\xc2' in position 30: ordinal not in range(128)

In [9]:
dfg.to_csv(r'C:\\Users\\adishree.b\\Downloads\\checkfile.csv', index=False)

UnicodeEncodeError: 'ascii' codec can't encode character u'\xc2' in position 30: ordinal not in range(128)